In [ ]:
%load_ext autoreload
%autoreload 2

import plotly.express as px
import pandas as pd
import numpy as np
from IPython.display import display

from duet.process import *
from duet.constants import *
from common import *

df_raw = load_raw()
unique_suites = df_raw[RF.suite].unique()
df_raw.describe()

In [ ]:
df = df_raw[
    (
        (df_raw["suite"].isin(["renaissance"]))
        & (df_raw["benchmark"].isin(["chi-square"]))
        & (df_raw["runid"].isin([0]))
        # & (df_raw["pair"].isin(["B"]))
        & (df_raw["type"].isin(["duet"]))
        & (df_raw["iteration"] <= 10)
        # & (df_raw["hostname"].isin(["cirrus-1.graal.d3s.hide.ms.mff.cuni.cz"]))
        # (df_raw[AF.hostname] == df_raw[AF.hostname].iloc[0])
        # & (
        #    df_raw["iteration_start"]
        #    < df_raw[RF.start].min() + pd.DateOffset(minutes=10)
        # )
    )
]

convert_ns(df)[ITER_ID_COL + TIME_D_COL]

In [ ]:
df2 = alter_score(df, rate=0.2)
convert_ns(df2)[ITER_ID_COL + TIME_D_COL]

px.timeline(
    df2,
    x_start=RF.start,
    x_end=RF.end,
    y=RF.pair,
    color=RF.type,
    hover_data=[RF.benchmark, RF.time, RF.pair, RF.runid],
).show()